In [14]:
%pip install fastembed "qdrant-client[fastembed]>=1.14.2" sentence-transformers huggingface_hub[hf_xet] ipywidgets -q

Note: you may need to restart the kernel to use updated packages.


In [15]:
from fastembed import TextEmbedding

In [16]:

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
import numpy as np
import requests



In [17]:
query = 'I just discovered the course. Can I join now?'
model_name = 'jinaai/jina-embeddings-v2-small-en'

model = TextEmbedding(model_name=model_name)
embeddings_query = list(model.embed([query]))
len(embeddings_query[0])

512

In [18]:
round(np.min(embeddings_query[0]), 3)


np.float64(-0.117)

In [19]:

np.linalg.norm(embeddings_query[0]), embeddings_query[0].dot(embeddings_query[0])

(np.float64(1.0), np.float64(1.0000000000000002))

In [20]:
doc = 'Can I still join the course after the start date?'
embeddings_doc = list(model.embed([doc]))
embeddings_doc[0].dot(embeddings_query[0])

np.float64(0.9008528895674548)

In [21]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
              'section': 'General course-related questions',
              'question': 'Course - Can I still join the course after the start date?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
              'section': 'General course-related questions',
              'question': 'Course - Can I follow the course after it finishes?',
              'course': 'data-engineering-zoomcamp'},
             {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
              'section': 'General course-related questions',
              'question': 'Course - When will the course start?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
              'section': 'General course-related questions',
              'question': 'Course - What can I do before the course starts?',
              'course': 'data-engineering-zoomcamp'},
             {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
              'section': 'General course-related questions',
              'question': 'How can we contribute to the course?',
              'course': 'data-engineering-zoomcamp'}
            ]

embeddings_vector = []

for document in documents:
    embeddings_doc = list(model.embed([document['text']]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmin(cos_sim)

np.int64(3)

In [22]:
embeddings_vector = []

for document in documents:
    full_text = document['question'] + ' ' + document['text']
    embeddings_doc = list(model.embed([full_text]))
    embeddings_vector.append(embeddings_doc[0])

cos_sim = np.array(embeddings_vector).dot(embeddings_query[0])
np.argmax(cos_sim)

np.int64(0)

In [23]:
models_list = TextEmbedding.list_supported_models()

dimensions = []
for model in models_list:
    dimensions.append(model['dim'])
    
models_list[np.array(dimensions).argmin()]

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

In [24]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
documents[:5]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [26]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer

# Initialize Qdrant client
client = QdrantClient("http://localhost:6333")

collection_name = "llmzoomcamp-rag"
EMBEDDING_DIMENSIONALITY = 384

# Create collection if it doesn't exist
if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=EMBEDDING_DIMENSIONALITY,
            distance=Distance.COSINE
        )
    )

# Load embedding model
model_name = 'BAAI/bge-small-en'
embedder = SentenceTransformer(model_name)

# Prepare and embed documents
points = []
for id_, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = embedder.encode(text).tolist()  # Convert NumPy array to list
    point = PointStruct(
        id=id_,
        vector=vector,
        payload={
            "text": text,
            "section": doc['section'],
            "course": doc['course']
        }
    )
    points.append(point)

# Upsert into Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)